# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — я её уже сделал.

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке.

## Откройте и изучите файл

Импортируем необходимые данные

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.dummy import DummyClassifier
import warnings

In [2]:
users_behavior = pd.read_csv('D:\Женя\Обучение Женя\Datasets/users_behavior.csv')
users_behavior.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Проверим типы данных

In [3]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.6 KB


В столбце messages поменяем тип данных на целочисленный

In [4]:
users_behavior['messages'] = users_behavior['messages'].astype('Int64')
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null Int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: Int64(1), float64(3), int64(1)
memory usage: 128.7 KB


In [5]:
users_behavior.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Видим информацию с разных ракурсов

Проверим данные на наличие дубликатов

In [6]:
users_behavior.duplicated().sum()

0

Дубликатов не найдено

Даные были проверены на предмет пропусков, дубликатов. С данными никаких аномалий не обнаружено, названия столбцов заданы корректно!

## Разбейте данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки.
Сначала разделим исходный датасет на обучающую и валидационную выборки в пропорции 60% и 40%. Затем разделим валидационную выборку пополам на валидационную и тестовую. 

In [7]:
users_behavior_train, users_behavior_valid = train_test_split(users_behavior, 
                                        train_size=0.60, test_size=0.40, random_state=123, stratify=users_behavior['is_ultra'])
users_behavior_valid, users_behavior_test = train_test_split(users_behavior_valid, 
                                       train_size = 0.50, test_size=0.50, random_state=123, stratify=users_behavior_valid['is_ultra'])

В каждой выборке выделим признаки (features) и целевой признак (target). В нашем случае целевой признак это данные из столбца is_ultra.

In [8]:
#обучающая выборка
features_train = users_behavior_train.drop(['is_ultra'], axis=1)
target_train = users_behavior_train['is_ultra']

#валидационная выборка
features_valid = users_behavior_valid.drop(['is_ultra'], axis=1)
target_valid = users_behavior_valid['is_ultra']

#тестовая выборка
features_test = users_behavior_test.drop(['is_ultra'], axis=1)
target_test = users_behavior_test['is_ultra']

## Исследуйте модели

# Random Forest

In [9]:
model_rf = RandomForestClassifier(random_state=123)
model_rf.fit(features_train, target_train)
predictions_rf = model_rf.predict(features_valid)
accuracy_rf = accuracy_score(target_valid, predictions_rf)
print('Result RF:', accuracy_rf)

Result RF: 0.7682737169517885


D:\Anakonda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Мы получили данный результат, но это не лучшмй результат, поскольку мы не настроили гиперпараметры. С помощью цикла найдем лучший результат. Для этого в переменной result_rf сохраним все результаты и из нее найдем максимальный. А результат без гиперпараметров сохраним в переменной without_parameters

In [10]:
without_hyperparameters = []
without_hyperparameters.append({'RandomForestClassifier':accuracy_rf})

In [11]:
result_rf =[]
for depth in range(1, 11):
    for est in range(10, 101, 10):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy_valid = accuracy_score(target_valid, predictions_valid)
        result_rf.append({'Model name': 'RandomForestClassifier',
                         'Hyperparameters': {'random_state':123,
                                            'n_estimators': est,
                                            'max_depth': depth},
                         'Accuracy score': accuracy_valid})

Сохраним лучший результат в переменной users_behavior_rf

In [12]:
users_behavior_rf = pd.DataFrame.from_dict(result_rf)
users_behavior_rf[users_behavior_rf['Accuracy score'] == users_behavior_rf['Accuracy score'].max()]

,Model name,Hyperparameters,Accuracy score
70,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 10, 'max...",0.802488
77,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 80, 'max...",0.802488


Наилучший результат достигнутый моделью - 0.835148 

Сохраним результат в переменной best_result

In [13]:
best_results =[]
best_results.append(users_behavior_rf.loc[86])

# DecisionTree

Также как и в предыдущем шаге попробуем получить результат без гиперпараметров

In [14]:
model_dt = DecisionTreeClassifier(random_state=123)
model_dt.fit(features_train, target_train)
predictions_valid = model_dt.predict(features_valid)
accuracy_dt = accuracy_score(target_valid, predictions_valid)
print('Result DT:', accuracy_dt)

Result DT: 0.6920684292379471


Сохраним результат в переменной without_parameters

In [15]:
without_hyperparameters.append({'DecisionTreeClassifier':accuracy_dt})

Для получения лучшего результата переберем в цикле гиперпараметр глубины деревьев(depth). Результат сохраним в переменной result_dt

In [16]:
result_dt = []
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)
    result_dt.append({'Model name': 'DecisionTreeClassifier',
                     'Hyperparameters': {'random_state': 123, 'max_depth':depth},
                                        'Accuracy score': accuracy_valid})

Выведим лучший результат

In [17]:
users_behavior_dt = pd.DataFrame.from_dict(result_dt)
users_behavior_dt[users_behavior_dt['Accuracy score'] == users_behavior_dt['Accuracy score'].max()]

,Model name,Hyperparameters,Accuracy score
7,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 8}",0.807154


Наилучший результат получается при параметрах max_depth: 5 и равен 0.822706

Сохраним результат в переменной best_result

In [18]:
best_results.append(users_behavior_dt.loc[4])

# LogisticRegression

Также проверим результат без гиперпараметров и сохраним в переменную without_parameters

In [19]:
model_lr_without_hyperparameters = LogisticRegression()
model_lr_without_hyperparameters.fit(features_train, target_train)
predictions_lr = model_lr_without_hyperparameters.predict(features_valid)
accuracy_lr = accuracy_score(target_valid, predictions_lr)
accuracy_lr

D:\Anakonda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6936236391912908

In [20]:
without_hyperparameters.append({'LogisticRegression':accuracy_lr})

Применим гиперпараметры

In [21]:
results_lr = []
model_lr = LogisticRegression(random_state=123, solver='lbfgs', penalty='l2')
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(target_valid, predictions_lr)
results_lr.append({'Model name': 'LogisticRegression',
                 'Hyperparameters': {'random_state': 123, 'solver':'lbfgs', 'penalty': 'l2'},
                 'Accuracy score': accuracy_lr})
accuracy_lr

0.6967340590979783

Попробуем применить другие гиперпараметры(solver='liblinear', penalty='l1')

In [22]:
model_lr = LogisticRegression(random_state=123, solver='liblinear', penalty='l1', C=0.001)
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(target_valid, predictions_lr)
results_lr.append({'Model name': 'LogisticRegression',
                 'Hyperparameters': {'random_state': 123, 'solver':'liblinear', 'penalty': 'l1'},
                 'Accuracy score': accuracy_lr})
accuracy_lr

0.6936236391912908

Результат лучше. Добавим его в best_results

In [23]:
users_behavior_lr = pd.DataFrame.from_dict(results_lr)
best_results.append(users_behavior_lr.loc[1])

# Сравнение моделей

Для начала сравним результаты расчетов без применения гиперпараметров

In [24]:
pd.DataFrame(without_hyperparameters)

,RandomForestClassifier,DecisionTreeClassifier,LogisticRegression
0,0.768274,NaN,NaN
1,NaN,0.692068,NaN
2,NaN,NaN,0.693624


Из таблицы видно, что лучший результат показал RandomForestClassifier. Худший DecisionTreeClassifier 

Посмотрим на лучшие расчеты с гиперпараметрами

In [25]:
pd.DataFrame(best_results)

,Model name,Hyperparameters,Accuracy score
86,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 70, 'max...",0.791602
4,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 5}",0.783826
1,LogisticRegression,"{'random_state': 123, 'solver': 'liblinear', '...",0.693624


Из данной таблицы видно, что лучший результат получился также методом RandomForestClassifier. Худший методом LogisticRegression

# Вывод

Были произведены расчеты путем применения трех методов - RandomForestClassifier, DecisionTreeClassifier и LogisticRegression. Каждую модель обучили на обучающей и валидационной выборках. Было выяснено, что без применения гиперпараметров лушчий результат показал метод RandomForestClassifier (метод Случайного леса). Также с настройкой гиперпараметров лидирует результат того же метода - RandomForestClassifier. 

Проверим модель RandomForestClassifier на тестовой выборке c гиперпараметрами(n_estimators=70, max_depth=9)

## Проверьте модель на тестовой выборке

In [26]:
model_test = RandomForestClassifier(random_state=123, n_estimators=70, max_depth=9)
model_test.fit(features_train, target_train)
predictions_test = model_test.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test)
accuracy_test

0.8195956454121306

Проверка модели показала, что модель способна правильно предсказать около 82% ответов.

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность используем DummyModels

In [27]:
dummy_final = DummyClassifier(strategy='most_frequent')
dummy_final.fit(features_train, target_train)
predictions_test = dummy_final.predict(features_test)
accuracy_final = accuracy_score(target_test, predictions_test)
print(accuracy_final)

0.6936236391912908
